### 1. MNIST 데이터 셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성

좋은 파라미터 값만 찾기

#### 데이터 셋 다운로드

In [1]:
from sklearn.datasets import fetch_openml

mnist_784 = fetch_openml("mnist_784", version=1, as_frame=False)

In [3]:
X, y = mnist_784["data"], mnist_784["target"]

데이터 셋을 동일한 비율로 분할

In [16]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state = 42)

In [121]:
for train_index, test_index in split.split(X, y):
    X_train = X[train_index]; y_train = y[train_index]
    X_test = X[test_index]; y_test = y[test_index]

In [122]:
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

In [59]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

In [11]:
from sklearn.model_selection import GridSearchCV

grid_param = [{"weights": ["uniform", "distance"],
            "n_neighbors": [3, 4, 5, 6, 7, 8]}]

grid_search = GridSearchCV(knn, grid_param, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5, 6, 7, 8],
                          'weights': ['uniform', 'distance']}],
             scoring='accuracy')

In [13]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.9703928571428572 {'n_neighbors': 3, 'weights': 'uniform'}
0.9715535714285716 {'n_neighbors': 3, 'weights': 'distance'}
0.9686785714285714 {'n_neighbors': 4, 'weights': 'uniform'}
0.9723928571428573 {'n_neighbors': 4, 'weights': 'distance'}
0.9690535714285714 {'n_neighbors': 5, 'weights': 'uniform'}
0.9705714285714284 {'n_neighbors': 5, 'weights': 'distance'}
0.9687321428571428 {'n_neighbors': 6, 'weights': 'uniform'}
0.9716428571428573 {'n_neighbors': 6, 'weights': 'distance'}
0.9685 {'n_neighbors': 7, 'weights': 'uniform'}
0.9696785714285714 {'n_neighbors': 7, 'weights': 'distance'}
0.967375 {'n_neighbors': 8, 'weights': 'uniform'}
0.9701607142857143 {'n_neighbors': 8, 'weights': 'distance'}


In [15]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9723928571428573
{'n_neighbors': 4, 'weights': 'distance'}


---

### 2. 데이터 증식으로 모델 성능이 높아지는지 확인

빠른 속도로 큰 데이터를 다루기 위해서는 리스트가 적합  
경우에 따라 넘파이와 리스트 간의 변환에 시간 많이 드므로(초 단위나 분 단위로 넘어갈 정도로 많은 시간이 듦) 사용할 데이터 타입(넘파이, 리스트)으로 바꿔두고 작업하는 것도 좋음  
  
아래 코드는 데이터 증강으로 더 많은 데이터를 추가해야 하므로 넘파이를 미리 리스트로 변환한 것임.

In [2]:
import numpy as np

X, y = mnist_784["data"].reshape(-1, 28, 28).tolist(), mnist_784["target"].tolist()

데이터 증강  
  
X에 증강된 데이터를 append하고, y에 증강된 데이터의 레이블을 append한다.

In [3]:
def move_digit(digit, direction):
    column_count = len(digit[0]); zero_row = [0 for i in range(column_count)]
    
    if direction == "left":
        for digit_row in digit:
            digit_row.pop(0)
            digit_row.append(0)
    if direction == "right":
        for digit_row in digit:
            digit_row.pop()
            digit_row.insert(0, 0)
    if direction == "up":
        digit.pop(0)
        digit.append(zero_row)
    if direction == "down":
        digit.pop()
        digit.insert(0, zero_row)
    
    return digit


In [4]:
for i, X_ele in enumerate(X.copy()):
    X.append(move_digit(X_ele, "left")); y.append(y[i])
    X.append(move_digit(X_ele, "right")); y.append(y[i])
    X.append(move_digit(X_ele, "up")); y.append(y[i])
    X.append(move_digit(X_ele, "down")); y.append(y[i])

데이터 분할(훈련 데이터, 테스트 데이터)

KNN은 2차원 데이터로 훈련

In [13]:
X = np.array(X).reshape(-1, 784); y = np.array(y)

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
    X_train = X[train_index]; y_train = y[train_index]
    X_test = X[test_index]; y_test = y[test_index]

KNN 모델 훈련 및 평가

In [15]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(weights="distance", n_neighbors=4)

In [16]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [17]:
knn.score(X_test, y_test)

1.0

### 3. 타이타닉 데이터 셋 in Kaggle

[캐글 타이타닉 데이터 셋](https://www.kaggle.com/competitions/titanic)